In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import sqlite3
import pandas as pd
from pathlib import Path
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import NearestNeighbors
import joblib

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Cargar los datos de las películas y las calificaciones
def load_data(db_path):
    db_path = Path(db_path)
    conn = sqlite3.connect(db_path)
    df_movies = pd.read_sql_query("SELECT * FROM movies", conn)
    df_ratings = pd.read_sql_query("SELECT * FROM ratings", conn)
    conn.close()
    # Fusionar las calificaciones de las películas con los datos de las películas
    df = pd.merge(df_ratings, df_movies, on='movieId', how='left')
    return df, df_movies

df, df_movies = load_data('/content/drive/MyDrive/bd_movies_recommendation/data/db_movies.sqlite')

In [ ]:
# Procesar año de estreno, géneros y añadir estadísticas
def extract_year(df):
    df['year'] = df['title'].str.extract(r'\((\d{4})\)').astype(float)
    return df

def process_genres(df):
    df['genres_list'] = df['genres'].str.split('|')  # Convertir géneros en listas
    df_genres = df['genres_list'].str.join('|').str.get_dummies(sep='|')  # Crear columnas dummy para cada género
    df = pd.concat([df, df_genres], axis=1)  # Concatenar las columnas de géneros
    return df

def add_statistics(df):
    # Añadir la calificación promedio de cada película
    movie_avg = df.groupby('movieId')['rating'].mean().rename('movie_avg_rating')
    df = df.merge(movie_avg, on='movieId', how='left')

    # Añadir la calificación promedio de cada usuario
    user_avg = df.groupby('userId')['rating'].mean().rename('user_avg_rating')
    df = df.merge(user_avg, on='userId', how='left')
    return df



In [ ]:
# Aplicar las funciones de procesamiento
df = extract_year(df)
df = process_genres(df)
df = add_statistics(df)

In [ ]:
# Escalar las calificaciones para el modelo KNN
scaler = StandardScaler()
df['rating_scaled'] = scaler.fit_transform(df[['rating']])

In [ ]:
# Crear una matriz de usuarios y películas para el modelo KNN
df_pivot = df.pivot(index='userId', columns='movieId', values='rating_scaled').fillna(0)


In [ ]:
# Entrenar el modelo KNN
knn = NearestNeighbors(n_neighbors=10, metric='cosine', algorithm='brute')
knn.fit(df_pivot.T)  # Transponer la matriz para que las películas sean las "instancias"

NearestNeighbors(algorithm='brute', metric='cosine', n_neighbors=10)

In [ ]:
# Modelo 1: Filtrado basado en vecinos (KNN)
# ------------------------------
# Este modelo recomienda películas basadas en la similitud entre las películas.
# Utiliza el algoritmo KNN para encontrar las películas más similares a una dada.
def recommend_movies_knn(movie_id, n_recommendations=10):
    # Localizar el índice de la película
    movie_idx = df_pivot.columns.get_loc(movie_id)

    # Obtener las películas más similares utilizando KNN
    distances, indices = knn.kneighbors(df_pivot.iloc[:, movie_idx].values.reshape(1, -1), n_neighbors=n_recommendations)

    # Obtener los IDs de las películas recomendadas
    recommended_movie_ids = df_pivot.columns[indices.flatten()]

    # Mapear los movieId a los títulos de las películas
    movie_titles = df_movies.set_index('movieId').loc[recommended_movie_ids]['title'].values
    return movie_titles


In [ ]:
# Modelo 2: Recomendación por Popularidad
# ------------------------------
# Este modelo recomienda las películas más populares basándose en el número de veces que han sido vistas o calificadas.
def recommend_movies_popularity(n_recommendations=10):
    # Contar cuántas veces ha sido calificada cada película
    movie_counts = df['movieId'].value_counts()

    # Seleccionar las 10 películas más populares
    top_movie_ids = movie_counts.head(n_recommendations).index
    movie_titles = df_movies.set_index('movieId').loc[top_movie_ids]['title'].values
    return movie_titles


In [ ]:
# Modelo 3: Recomendación basada en Promedio de Calificaciones
# ------------------------------
# Este modelo recomienda las películas con el mejor promedio de calificación entre todos los usuarios.
def recommend_movies_avg_rating(n_recommendations=10):
    # Calcular el promedio de calificación de cada película
    avg_ratings = df.groupby('movieId')['rating'].mean()

    # Seleccionar las 10 películas con las mejores calificaciones
    top_movie_ids = avg_ratings.sort_values(ascending=False).head(n_recommendations).index
    movie_titles = df_movies.set_index('movieId').loc[top_movie_ids]['title'].values
    return movie_titles


In [ ]:
# Modelo 4: Recomendación basada en Géneros Comunes
# ------------------------------
# Este modelo recomienda películas que comparten géneros con una película dada.
def recommend_movies_genres(movie_id, n_recommendations=10):
    # Obtener los géneros de la película seleccionada
    target_genres = df_movies.set_index('movieId').loc[movie_id]['genres']
    target_genres_set = set(target_genres.split('|'))

    # Filtrar las películas que comparten al menos un género
    df_movies['genre_match_count'] = df_movies['genres'].apply(lambda x: len(set(x.split('|')).intersection(target_genres_set)))

    # Filtrar las películas que comparten géneros y ordenar por el número de coincidencias
    recommended_movies = df_movies[df_movies['genre_match_count'] > 0].sort_values(by='genre_match_count', ascending=False).head(n_recommendations)
    return recommended_movies['title'].values

In [ ]:
# Ejemplo de uso de los modelos:
print("Recomendaciones KNN para la película con ID 1:")
print(recommend_movies_knn(1, n_recommendations=10))

print("\nRecomendaciones por popularidad (top 10 películas más vistas):")
print(recommend_movies_popularity(n_recommendations=10))

print("\nRecomendaciones basadas en el promedio de calificaciones (top 10):")
print(recommend_movies_avg_rating(n_recommendations=10))

print("\nRecomendaciones basadas en géneros comunes a la película con ID 1:")
print(recommend_movies_genres(1, n_recommendations=10))

# Guardar el modelo de recomendación KNN
joblib.dump(knn, '/content/drive/MyDrive/bd_movies_recommendation/models/knn_movie_recommender.pkl')
print("\nModelo de recomendación KNN guardado.")

Recomendaciones KNN para la película con ID 1:
['Toy Story (1995)' 'Toy Story 2 (1999)' 'Aladdin (1992)'
 'Finding Nemo (2003)' 'Incredibles, The (2004)' 'Toy Story 3 (2010)'
 'Monsters, Inc. (2001)' 'Lion King, The (1994)'
 'Wallace & Gromit: The Wrong Trousers (1993)' 'Back to the Future (1985)']

Recomendaciones por popularidad (top 10 películas más vistas):
['Forrest Gump (1994)' 'Shawshank Redemption, The (1994)'
 'Pulp Fiction (1994)' 'Silence of the Lambs, The (1991)'
 'Matrix, The (1999)' 'Star Wars: Episode IV - A New Hope (1977)'
 'Jurassic Park (1993)' 'Braveheart (1995)'
 'Terminator 2: Judgment Day (1991)' "Schindler's List (1993)"]

Recomendaciones basadas en el promedio de calificaciones (top 10):
["Won't You Be My Neighbor? (2018)" 'Jane Eyre (1944)' 'Rain (2001)'
 'Goodbye Charlie (1964)' 'Sorority House Massacre (1986)'
 'Slumber Party Massacre III (1990)' 'Slumber Party Massacre II (1987)'
 'True Stories (1986)' 'Moonlight' 'Tom Segura: Mostly Stories (2016)']

Recom

In [ ]:
# Guardar el modelo de recomendación KNN
joblib.dump(knn, '/content/drive/MyDrive/bd_movies_recommendation/models/knn_movie_recommender.pkl')
print("\nModelo de recomendación KNN guardado.")


Modelo de recomendación KNN guardado.
